<a href="https://colab.research.google.com/github/NeuralNetGeek/ml-monkeypox/blob/main/Monkey_Pox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, RocCurveDisplay

In [69]:
cols=['Patient_ID', 'Systemic Illness', 'Rectal Pain', 'Sore Throat', 'Penile Oedema', 'Oral Lesions', 'Solitary Lesion', 'Swollen Tonsils', 'HIV Infection', 'Sexually Transmitted Infection', 'MonkeyPox']
df=pd.read_csv('monkey.data.csv', header=None)
df.columns=cols
df.drop(['Patient_ID', 'Systemic Illness'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)

df['Lesions']=df[['Oral Lesions', 'Solitary Lesion']].any(axis=1).astype(int)
df['Swelling']=df[['Swollen Tonsils', 'Penile Oedema']].any(axis=1).astype(int)
df['Infections']=df[['Sexually Transmitted Infection', 'HIV Infection']].any(axis=1).astype(int)
df['Pain']=df[['Rectal Pain', 'Sore Throat']].any(axis=1).astype(int)

df['MonkeyPox_encoded']=df['MonkeyPox'].map({'Positive': 1, 'Negative': 0})

columns_to_drop=['Oral Lesions', 'Solitary Lesion', 'Swollen Tonsils', 'Penile Oedema', 'Rectal Pain', 'Sore Throat', 'Sexually Transmitted Infection', 'HIV Infection', 'MonkeyPox']
df.drop(columns=columns_to_drop, inplace=True)

features_encoded=['Lesions', 'Swelling', 'Infections', 'Pain']

df['MonkeyPox_encoded'].value_counts()

,count
MonkeyPox_encoded,
0,256
1,256


In [70]:
features_encoded = ['Lesions', 'Swelling', 'Infections', 'Pain']
target = 'MonkeyPox_encoded'

X = df[features_encoded]
y = df[target]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [71]:
models = {"Logistic Regression": LogisticRegression(), "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5), "Naive Bayes": GaussianNB(), "Support Vector Machine": SVC(probability=True), "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42), "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42)}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_classifiers(models, X, y, show_reports=True):
    summary = []

    for name, model in models.items():
        try:
            cv_scores = cross_val_score(model, X, y, cv=skf, scoring='f1')
            mean_cv_score = cv_scores.mean()

            model.fit(X, y)
            y_pred = model.predict(X)

            acc = accuracy_score(y, y_pred)
            prec = precision_score(y, y_pred, zero_division=0)
            rec = recall_score(y, y_pred, zero_division=0)
            f1 = f1_score(y, y_pred, zero_division=0)

            roc_auc = np.nan
            if hasattr(model, "predict_proba"):
                y_prob = model.predict_proba(X)[:, 1]
                if len(np.unique(y)) > 1:
                    roc_auc = roc_auc_score(y, y_prob)

            summary.append({
                'Model': name,
                'Accuracy': acc,
                'Precision': prec,
                'Recall': rec,
                'F1 Score': f1,
                'ROC-AUC': roc_auc,
                'Cross-Validation F1 Mean': mean_cv_score
            })

            if show_reports:
                print(f"\nModel: {name}")
                print(f"Cross-Validation F1 Mean: {mean_cv_score:.4f}")
                print("Confusion Matrix:")
                print(confusion_matrix(y, y_pred))
                print("\nClassification Report:")
                print(classification_report(y, y_pred, zero_division=0))

        except Exception as e:
            print(f"\nError with {name}: {e}")
            summary.append({
                'Model': name,
                'Accuracy': np.nan,
                'Precision': np.nan,
                'Recall': np.nan,
                'F1 Score': np.nan,
                'ROC-AUC': np.nan,
                'Cross-Validation F1 Mean': np.nan
            })

    return pd.DataFrame(summary)

In [72]:
results = evaluate_classifiers(models, X_train, y_train)
print("\n Model Comparison Summary:")
print(results)




Model: Logistic Regression
Cross-Validation F1 Mean: 0.3347
Confusion Matrix:
[[99 54]
 [88 65]]

Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.65      0.58       153
           1       0.55      0.42      0.48       153

    accuracy                           0.54       306
   macro avg       0.54      0.54      0.53       306
weighted avg       0.54      0.54      0.53       306


Model: K-Nearest Neighbors
Cross-Validation F1 Mean: 0.5674
Confusion Matrix:
[[ 40 113]
 [ 34 119]]

Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.26      0.35       153
           1       0.51      0.78      0.62       153

    accuracy                           0.52       306
   macro avg       0.53      0.52      0.49       306
weighted avg       0.53      0.52      0.49       306


Model: Naive Bayes
Cross-Validation F1 Mean: 0.3347
Confusion Matrix:
[[114  39]
 [105  48]]

Cla